# Наивный Байесовский классификатор в 25 строк кода
Наивный Байесовский классификатор один из самых простых из алгоритмов классификации. Тем не менее, очень часто он работает не хуже, а то и лучше более сложных алгоритмов. Здесь я хочу поделиться кодом и описанием того, как это все работает.

Итак, для примера возьму задачу определения пола по имени. Конечно, чтобы определить пол можно создать большой список имен с метками пола. Но этот список в любом случае будет неполон. Для того, чтобы решить эту проблему, можно "натренировать" модель по маркированными именам.

## Немного теории
Пусть у нас есть строка с текстом О. Кроме того, имеются классы С, к одному из которых мы должны отнести строку. Нам необходимо найти такой класс С, при котором его вероятность для данной строки была бы максимальна. Математически это записывается так:

$$ c = \arg \max_C P(C|O) $$

Вычислить Р(С|O) сложно. Но можно воспользоваться теоремой Байеса и перейти к косвенным вероятностям:

$$ P(C|O) = \frac{P(O|C)P(C)}{P(O)} $$

Так как мы ищем максимум от функции, то знаменатель нас не интересует (он в данном случае константа). Кроме того, нужно взглянуть на строку О. Обычно, нет смысла работать со всей строкой. Намного эффективнее выделить из нее определенные признаки(*features*). Таким образом формула примет вид:

$$ P(C|o_1 o_2...o_n) = \frac{P(o_1 o_2...o_n|C)P(C)}{P(o_1 o_2...o_n)} $$

Так как знаменатель нас не интересует, работаем с числителем. Здесь включаем "наивное" предположение о том, что переменные О зависят только от класса С, и не зависят друг от друга. Это сильное упрощение, но зачастую это работает. Числитель примет вид:

$$ P(C)P(o_1|C)P(o_2|C_{o_1})...P(o_n|C_{{o_1}{o_2}...{o_n}}) = p(C)P(o_1|C)P(o_2|C)...P(o_n|C) = P(C)\prod\limits_i(o_i|C) $$

Финальная формула примет вид:

$$ c = \arg \max_{c\in C}P(c|o_1o_2...o_n) = \arg \max_{c\in C}P(c)\prod\limits P(o_i|c)(1) $$

Т.е. все что нужно сделать, это вычислить вероятность Р(С) и Р(О|C). Вычисление этих параметров и называется тренировкой классификатора

## Код
Создаем две функции: одна для тренировки (подсчета параметров формулы), другая для классификации (непосредственный расчет формулы).

In [8]:
from __future__ import division
from collections import defaultdict
from math import log

In [10]:
def train(samples):
    classes, freq = defaultdict(lambda:0), defaultdict(lambda:0)
    for feats, label in samples:
        classes[label] += 1
        for feat in feats:
            freq[label, feat] += 1
            
    for label, feat in freq:
        freq[label, feat] /= classes[label]
    for c in classes:
        classes[c] /= len(samples)
    
    return classes, freq

In [14]:
def classify(classifier, feats):
    classes, prob = classifier
    return min(classes.keys(), 
        key = lambda cl: -log(classes[cl]) + \
            sum(-log(prob.get((cl,feat), 10**(-7))) for feat in feats))

В функции ```train``` первые пять строк производят подсчет количества классов С, а также частоту появления фич О и С в одном семпле. Вторая часть метода просто нормирует эти частоты. Таким образом на выходе получаются вероятности Р(С) и Р(О|C). 

В функции ```classify``` происходит поиск наиболее вероятного класса. Единственное отличие от формулы(1) в том, что я заменяю произведение вероятностей на сумму логарифмов, взятых с отрицательным знаком, и вычисляю не argmax, а argmin. Переход к логарифмам - распространенный прием чтобы избежать слишком маленьких чисел, которые могли бы получится при произведении вероятностей.

Число 10^(-7), которое представляется в логарифм, это способ избежать нуля в аргументе логарифма(т.к. он будет иначе неопределен).

Чтобы натренировать классификатор возьмем размеченный список мужских и женских имен и воспользуемся этим кодом:

In [ ]:
def get_features(sample): return (sample[-1],)

samples = (line.decode('utf-8').split() for line in open('names.txt'))
features